# 🎯 Comprehensive MNIST Embedding Comparison

This notebook compares the performance of different time embedding approaches on MNIST:
1. **Baseline LSTM** - No time embedding (raw pixel positions)
2. **LSTM + LETE** - With Learning Time Embedding (LeTE)
3. **LSTM + KAN-MAMMOTE** - With Improved KAN-MAMMOTE embedding

## 📊 Key Metrics to Compare:
- **Accuracy**: Classification performance
- **Training Speed**: Time per epoch
- **Parameter Count**: Model complexity
- **Convergence**: Training stability
- **Temporal Modeling**: How well each method captures temporal patterns

In [ ]:
# ============================================================================
# 📦 IMPORTS AND SETUP
# ============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Import our models
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

from src.models import KAN_MAMMOTE_Model, ImprovedKANMAMOTE  # Improved version as default
from src.LETE.LeTE import CombinedLeTE
from src.utils.config import KANMAMOTEConfig

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name()}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✅ All imports successful!")

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

## 📁 Data Setup

We'll convert MNIST images to event-based sequences where each non-zero pixel becomes an event with:
- **Timestamp**: Pixel position (row * width + col)
- **Features**: Pixel intensity (optional)
- **Label**: Digit class (0-9)

In [ ]:
# ============================================================================
# 🎲 EVENT-BASED MNIST DATASET
# ============================================================================

import sys
import os
import math  # Added missing import

# Add the src directory to Python path
sys.path.append('/mnt/c/Users/peera/Desktop/KAN-MAMMOTE/src')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchvision.datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Import our models - Fixed import path
from src.models import KAN_MAMMOTE_Model, ImprovedKANMAMOTE
from src.utils.config import KANMAMOTEConfig

print("📦 All imports successful!")
print(f"🔧 Using device: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}")

class EventBasedMNIST(Dataset):
    """
    Convert MNIST images to event-based sequences.
    Each non-zero pixel becomes an event with timestamp = pixel position.
    Based on EventBasedMNIST_with_log.ipynb implementation.
    """
    
    def __init__(self, root='./data', train=True, threshold=0.9, transform=None, download=True):
        """
        Args:
            root: Data directory
            train: Training or test set
            threshold: Minimum pixel intensity to consider as event
            transform: Image transformations
            download: Whether to download MNIST
        """
        self.root = root
        self.train = train
        self.threshold = threshold
        self.transform = transform
        
        # Load MNIST dataset (following EventBasedMNIST_with_log.ipynb pattern)
        if transform is None:
            transform = transforms.ToTensor()
        
        # Fixed: Use full torchvision.datasets path instead of just datasets
        self.data = torchvision.datasets.MNIST(
            root=self.root, 
            train=self.train, 
            transform=transform, 
            download=download
        )
        
        # Pre-process all images to event sequences
        self.event_data = []
        self.labels = []
        
        print(f"📊 Processing {'training' if train else 'test'} set to events...")
        
        for img, label in tqdm(self.data, desc="Converting to events"):
            # Flatten image to 1D (784 pixels for 28x28)
            img_flat = img.view(-1)  # (784,)
            
            # Find pixels above threshold (events)
            events = torch.nonzero(img_flat > self.threshold).squeeze()
            
            # Handle edge cases
            if events.dim() == 0:  # Single event
                events = events.unsqueeze(0)
            elif len(events) == 0:  # No events
                events = torch.tensor([0])  # Add dummy event
                
            # Sort events by position (timestamp order)
            events = torch.sort(events).values
            
            self.event_data.append(events)
            self.labels.append(label)
        
        print(f"✅ Processed {len(self.event_data)} samples")
        print(f"   Average events per sample: {sum(len(events) for events in self.event_data) / len(self.event_data):.1f}")
        
    def __len__(self):
        return len(self.event_data)
    
    def __getitem__(self, idx):
        events = self.event_data[idx]
        label = self.labels[idx]
        
        # Create features based on event positions
        # For compatibility with our models, we extract pixel intensities
        if len(events) > 0:
            # Get original image to extract intensities
            original_img, _ = self.data[idx]
            img_flat = original_img.view(-1)
            
            # Extract intensities for the events
            intensities = img_flat[events]
            features = intensities.unsqueeze(1)  # (seq_len, 1)
        else:
            # Handle empty case
            features = torch.zeros(1, 1)
            
        return events, features, len(events), label

def collate_fn(batch):
    """
    Custom collate function for variable-length sequences.
    Compatible with EventBasedMNIST_with_log.ipynb approach.
    """
    events_list = []
    features_list = []
    lengths = []
    labels_list = []
    
    for events, features, length, label in batch:
        events_list.append(events)
        features_list.append(features)
        lengths.append(length)
        labels_list.append(label)
    
    # Pad sequences
    padded_events = pad_sequence(events_list, batch_first=True, padding_value=0)
    padded_features = pad_sequence(features_list, batch_first=True, padding_value=0.0)
    
    lengths = torch.tensor(lengths, dtype=torch.long)
    labels = torch.tensor(labels_list, dtype=torch.long)
    
    return padded_events, padded_features, lengths, labels

# Create datasets (matching EventBasedMNIST_with_log.ipynb parameters)
print("🎲 Creating Event-Based MNIST datasets...")
train_dataset = EventBasedMNIST(root='./data', train=True, threshold=0.9, download=True)
test_dataset = EventBasedMNIST(root='./data', train=False, threshold=0.9, download=True)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

print(f"📦 Data loaders created:")
print(f"   Train: {len(train_loader)} batches")
print(f"   Test: {len(test_loader)} batches")

# Test data loading
sample_batch = next(iter(train_loader))
events, features, lengths, labels = sample_batch
print(f"\n📋 Sample batch:")
print(f"   Events shape: {events.shape}")
print(f"   Features shape: {features.shape}")
print(f"   Lengths: {lengths[:5]}")
print(f"   Labels: {labels[:5]}")
print(f"   Events range: [{events.min()}, {events.max()}]")
print(f"   Average sequence length: {lengths.float().mean():.1f}")

📦 All imports successful!
🔧 Using device: NVIDIA GeForce RTX 4060 Laptop GPU
🎲 Creating Event-Based MNIST datasets...
📊 Processing training set to events...
📊 Processing training set to events...


Converting to events: 100%|██████████| 60000/60000 [00:05<00:00, 10252.72it/s]



✅ Processed 60000 samples
   Average events per sample: 68.6
📊 Processing test set to events...
📊 Processing test set to events...


Converting to events: 100%|██████████| 10000/10000 [00:00<00:00, 14068.84it/s]

✅ Processed 10000 samples
   Average events per sample: 70.3
📦 Data loaders created:
   Train: 938 batches
   Test: 157 batches

📋 Sample batch:
   Events shape: torch.Size([64, 159])
   Features shape: torch.Size([64, 159, 1])
   Lengths: tensor([ 40,  73, 159,  67,  79])
   Labels: tensor([1, 2, 8, 5, 2])
   Events range: [0, 766]
   Average sequence length: 76.3


## 🏗️ Model Definitions

We'll define three different LSTM-based models:
1. **Baseline LSTM**: Raw timestamps → LSTM → Classifier
2. **LSTM + LETE**: Timestamps → LETE → LSTM → Classifier
3. **LSTM + KAN-MAMMOTE**: Timestamps → KAN-MAMMOTE → LSTM → Classifier

In [ ]:
# ============================================================================
# 🔧 STANDARDIZED CONFIGURATION FOR FAIR COMPARISON
# ============================================================================

import math

# Standard configuration for all models
STANDARD_CONFIG = {
    'lstm_hidden_dim': 128,     # Same LSTM hidden dimension for all models
    'lstm_num_layers': 2,       # Same LSTM layers for all models
    'lstm_dropout': 0.2,        # Same LSTM dropout for all models
    'time_emb_dim': 32,         # Standardized time embedding dimension
    'num_classes': 10           # MNIST classes
}

print("🔧 STANDARDIZED CONFIGURATION FOR FAIR COMPARISON:")
print(f"   LSTM Hidden Dim: {STANDARD_CONFIG['lstm_hidden_dim']}")
print(f"   LSTM Layers: {STANDARD_CONFIG['lstm_num_layers']}")
print(f"   LSTM Dropout: {STANDARD_CONFIG['lstm_dropout']}")
print(f"   Time Embedding Dim: {STANDARD_CONFIG['time_emb_dim']}")
print(f"   Output Classes: {STANDARD_CONFIG['num_classes']}")
print("✅ All models will use identical LSTM architectures!")

🔧 STANDARDIZED CONFIGURATION FOR FAIR COMPARISON:
   LSTM Hidden Dim: 128
   LSTM Layers: 2
   LSTM Dropout: 0.2
   Time Embedding Dim: 32
   Output Classes: 10
✅ All models will use identical LSTM architectures!


In [ ]:
# ============================================================================
# 🎯 MODEL 1: BASELINE LSTM (STANDARDIZED)
# ============================================================================

class StandardizedBaselineLSTM(nn.Module):
    """
    STANDARDIZED Baseline LSTM model with simple temporal information.
    Uses the same LSTM architecture as all other models for fair comparison.
    """
    
    def __init__(self, config=STANDARD_CONFIG):
        super().__init__()
        self.config = config
        
        # Input: [normalized_timestamp, pixel_intensity] = 2 dimensions
        input_dim = 2
        
        # STANDARDIZED LSTM (identical to all other models)
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=config['lstm_hidden_dim'],
            num_layers=config['lstm_num_layers'],
            batch_first=True,
            dropout=config['lstm_dropout']
        )
        
        # STANDARDIZED classifier
        self.classifier = nn.Linear(config['lstm_hidden_dim'], config['num_classes'])
        
    def forward(self, events, features, lengths):
        # Combine timestamps and features
        # events: (batch, seq_len) - timestamps
        # features: (batch, seq_len, 1) - pixel intensities
        
        # Normalize timestamps to [0, 1] range
        timestamps_normalized = (events.float() / 783.0).unsqueeze(-1)  # (batch, seq_len, 1)
        
        # Combine timestamp and pixel intensity
        combined_input = torch.cat([timestamps_normalized, features], dim=-1)  # (batch, seq_len, 2)
        
        # Pack sequences for LSTM
        packed = pack_padded_sequence(combined_input, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # STANDARDIZED LSTM forward pass
        lstm_out, (h_n, c_n) = self.lstm(packed)
        
        # Use last hidden state for classification
        final_hidden = h_n[-1]  # (batch, lstm_hidden_dim)
        
        # Classify
        logits = self.classifier(final_hidden)
        return logits

# ============================================================================
# 🌟 MODEL 2: LSTM + SinCos (STANDARDIZED)
# ============================================================================

class StandardizedSinCosEmbedding(nn.Module):
    """
    STANDARDIZED SinCos embedding with consistent dimensions.
    """
    def __init__(self, d_model=32, max_len=784):
        super().__init__()
        self.d_model = d_model
        
        # Create FIXED sinusoidal embeddings (non-learnable)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Register as buffer (not parameter) - fixed embeddings
        self.register_buffer('pe', pe)
        
    def forward(self, timestamps):
        """
        Args:
            timestamps: (batch, seq_len) - pixel positions [0, 783]
        Returns:
            time_emb: (batch, seq_len, d_model)
        """
        batch_size, seq_len = timestamps.shape
        
        # Normalize pixel positions to valid range [0, 783]
        timestamps_norm = torch.clamp(timestamps.long(), 0, 783)
        
        # Get fixed sinusoidal embeddings
        time_emb = self.pe[timestamps_norm]  # (batch, seq_len, d_model)
        
        return time_emb

class StandardizedLSTM_SinCos(nn.Module):
    """STANDARDIZED LSTM model with SinCos time embeddings."""
    
    def __init__(self, config=STANDARD_CONFIG):
        super().__init__()
        self.config = config
        
        # STANDARDIZED SinCos time embedding
        self.time_embedding = StandardizedSinCosEmbedding(d_model=config['time_emb_dim'])
        
        # Feature processing (project to match time embedding dimension)
        self.feature_projection = nn.Linear(1, config['time_emb_dim'])
        
        # STANDARDIZED LSTM (identical to all other models)
        lstm_input_dim = config['time_emb_dim'] + config['time_emb_dim']  # time_emb + feature_proj
        self.lstm = nn.LSTM(
            input_size=lstm_input_dim,
            hidden_size=config['lstm_hidden_dim'],
            num_layers=config['lstm_num_layers'],
            batch_first=True,
            dropout=config['lstm_dropout']
        )
        
        # STANDARDIZED classifier
        self.classifier = nn.Linear(config['lstm_hidden_dim'], config['num_classes'])
        
        # Conservative weight initialization
        self._init_weights()
        
    def _init_weights(self):
        """Conservative weight initialization to prevent gradient issues."""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight, gain=0.5)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
            elif isinstance(module, nn.LSTM):
                for name, param in module.named_parameters():
                    if 'weight' in name:
                        nn.init.xavier_uniform_(param, gain=0.5)
                    elif 'bias' in name:
                        nn.init.zeros_(param)
        
    def forward(self, events, features, lengths):
        # Get STANDARDIZED SinCos time embeddings
        time_emb = self.time_embedding(events)  # (batch, seq_len, time_emb_dim)
        
        # Process features to match time embedding dimension
        feature_emb = self.feature_projection(features)  # (batch, seq_len, time_emb_dim)
        
        # Combine embeddings
        combined = torch.cat([time_emb, feature_emb], dim=-1)  # (batch, seq_len, 2*time_emb_dim)
        
        # Pack sequences for LSTM
        packed = pack_padded_sequence(combined, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # STANDARDIZED LSTM forward pass
        lstm_out, (h_n, c_n) = self.lstm(packed)
        
        # Use last hidden state for classification
        final_hidden = h_n[-1]  # (batch, lstm_hidden_dim)
        
        # Classify
        logits = self.classifier(final_hidden)
        return logits

# ============================================================================
# 🔥 MODEL 3: LSTM + LETE (STANDARDIZED)
# ============================================================================

class StandardizedLSTM_LETE(nn.Module):
    """
    STANDARDIZED LSTM model with LETE time embedding.
    """
    
    def __init__(self, config=STANDARD_CONFIG):
        super().__init__()
        self.config = config
        
        # Initialize LETE with STANDARDIZED embedding dimension
        self.use_lete = False
        try:
            # Use STANDARDIZED time embedding dimension
            self.time_encoder = CombinedLeTE(config['time_emb_dim'], p=0.5)
            
            # Test LETE with dummy data
            dummy_input = torch.randn(1, 5).abs().clamp(0, 783)
            with torch.no_grad():
                test_emb = self.time_encoder(dummy_input)
                if torch.isnan(test_emb).any() or torch.isinf(test_emb).any():
                    raise ValueError("LETE produces NaN/Inf on test input")
            
            self.use_lete = True
            print("✅ LETE embedding initialized successfully with standardized dimension")
            
        except Exception as e:
            print(f"❌ LETE initialization failed: {e}")
            # Fallback to simple embedding
            self.time_encoder = nn.Embedding(784, config['time_emb_dim'])
            self.use_lete = False
            print("⚠️ Using simple embedding as fallback")
        
        # STANDARDIZED LSTM (identical to all other models)
        self.lstm = nn.LSTM(
            input_size=config['time_emb_dim'],
            hidden_size=config['lstm_hidden_dim'],
            num_layers=config['lstm_num_layers'],
            batch_first=True,
            dropout=config['lstm_dropout']
        )
        
        # STANDARDIZED classifier
        self.classifier = nn.Linear(config['lstm_hidden_dim'], config['num_classes'])
        
    def forward(self, events, features, lengths):
        batch_size = events.size(0)
        
        if self.use_lete:
            # Use LETE with proper input handling
            events_float = events.float()
            
            try:
                # Apply LETE time encoding
                embedded = self.time_encoder(events_float)  # (batch, seq_len, time_emb_dim)
                
                # Check for NaN/Inf
                if torch.isnan(embedded).any() or torch.isinf(embedded).any():
                    print("⚠️ LETE produced NaN/Inf, using zero embedding")
                    embedded = torch.zeros(batch_size, events.size(1), self.config['time_emb_dim'], device=events.device)
                    
            except Exception as e:
                print(f"⚠️ LETE forward failed: {e}, using zero embedding")
                embedded = torch.zeros(batch_size, events.size(1), self.config['time_emb_dim'], device=events.device)
        else:
            # Use simple embedding fallback
            events_clamped = torch.clamp(events.long(), 0, 783)
            embedded = self.time_encoder(events_clamped)
        
        # Pack sequences for LSTM
        packed = pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # STANDARDIZED LSTM forward pass
        _, (h_n, c_n) = self.lstm(packed)
        
        # Use last hidden state for classification
        final_hidden = h_n[-1]  # (batch, lstm_hidden_dim)
        
        # Classify
        logits = self.classifier(final_hidden)
        
        return logits

# ============================================================================
# 🚀 MODEL 4: LSTM + KAN-MAMMOTE (STANDARDIZED)
# ============================================================================

class StandardizedLSTM_KAN_MAMMOTE(nn.Module):
    """
    STANDARDIZED LSTM model with KAN-MAMMOTE time embedding.
    """
    
    def __init__(self, config=STANDARD_CONFIG):
        super().__init__()
        self.config = config
        
        # KAN-MAMMOTE configuration with STANDARDIZED output dimension
        self.kan_config = KANMAMOTEConfig(
            D_time=config['time_emb_dim'],  # STANDARDIZED time embedding dimension
            num_experts=4,
            hidden_dim_mamba=config['time_emb_dim'],  # Match time embedding dimension
            state_dim_mamba=16,  # Smaller state dimension
            num_mamba_layers=2,
            gamma=0.3,
            use_aux_features_router=False,
            raw_event_feature_dim=0,
            K_top=2,
            # Faster-KAN parameters
            kan_grid_size=5,
            kan_grid_min=-2.0,
            kan_grid_max=2.0,
            kan_spline_scale=0.667,
            kan_num_layers=2,
            kan_hidden_dim=config['time_emb_dim']
        )
        
        # KAN-MAMMOTE for time embedding
        self.kan_mammote = ImprovedKANMAMOTE(self.kan_config)
        
        # Feature projection to match time embedding dimension
        self.feature_projection = nn.Linear(1, config['time_emb_dim'])
        
        # STANDARDIZED LSTM (identical to all other models)
        lstm_input_dim = config['time_emb_dim'] + config['time_emb_dim']  # kan_emb + feature_proj
        self.lstm = nn.LSTM(
            input_size=lstm_input_dim,
            hidden_size=config['lstm_hidden_dim'],
            num_layers=config['lstm_num_layers'],
            batch_first=True,
            dropout=config['lstm_dropout']
        )
        
        # STANDARDIZED classifier
        self.classifier = nn.Linear(config['lstm_hidden_dim'], config['num_classes'])
        
    def forward(self, events, features, lengths):
        batch_size = events.size(0)
        
        # Normalize timestamps to [0, 1] range for KAN-MAMMOTE
        timestamps = events.float() / 783.0
        timestamps = timestamps.unsqueeze(-1)  # (batch, seq_len, 1)
        
        # Empty features for KAN-MAMMOTE
        empty_features = torch.zeros(batch_size, timestamps.size(1), 0, device=timestamps.device)
        
        # Apply KAN-MAMMOTE embedding
        try:
            kan_emb, kan_info = self.kan_mammote(timestamps, empty_features)
            # kan_emb: (batch, seq_len, time_emb_dim)
        except Exception as e:
            print(f"KAN-MAMMOTE error: {e}")
            # Fallback: create embeddings with correct dimension
            kan_emb = torch.zeros(batch_size, timestamps.size(1), self.config['time_emb_dim'], device=timestamps.device)
            kan_info = {}
        
        # Project pixel features to match time embedding dimension
        feature_emb = self.feature_projection(features)  # (batch, seq_len, time_emb_dim)
        
        # Combine embeddings
        combined = torch.cat([kan_emb, feature_emb], dim=-1)  # (batch, seq_len, 2*time_emb_dim)
        
        # Pack sequences for LSTM
        packed = pack_padded_sequence(combined, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # STANDARDIZED LSTM forward pass
        lstm_out, (h_n, c_n) = self.lstm(packed)
        
        # Use last hidden state for classification
        final_hidden = h_n[-1]  # (batch, lstm_hidden_dim)
        
        # Classify
        logits = self.classifier(final_hidden)
        
        return logits, kan_info

# ============================================================================
# 🏗️ CREATE STANDARDIZED MODELS
# ============================================================================

print("🏗️ Creating standardized models with identical LSTM architectures...")

# Create all models with standardized configuration
baseline_model = StandardizedBaselineLSTM(STANDARD_CONFIG).to(device)
sincos_model = StandardizedLSTM_SinCos(STANDARD_CONFIG).to(device)
lete_model = StandardizedLSTM_LETE(STANDARD_CONFIG).to(device)
kan_model = StandardizedLSTM_KAN_MAMMOTE(STANDARD_CONFIG).to(device)

# Calculate parameters
baseline_params = sum(p.numel() for p in baseline_model.parameters() if p.requires_grad)
sincos_params = sum(p.numel() for p in sincos_model.parameters() if p.requires_grad)
lete_params = sum(p.numel() for p in lete_model.parameters() if p.requires_grad)
kan_params = sum(p.numel() for p in kan_model.parameters() if p.requires_grad)

print(f"\n📊 STANDARDIZED Model Parameter Comparison:")
print(f"   Baseline LSTM:        {baseline_params:,} parameters")
print(f"   LSTM + SinCos:        {sincos_params:,} parameters")
print(f"   LSTM + LETE:          {lete_params:,} parameters")
print(f"   LSTM + KAN-MAMMOTE:   {kan_params:,} parameters")

# Calculate LSTM-only parameters for comparison
lstm_only_params = (STANDARD_CONFIG['lstm_hidden_dim'] * 2 + 1) * STANDARD_CONFIG['lstm_hidden_dim'] * 4 * STANDARD_CONFIG['lstm_num_layers']
classifier_params = STANDARD_CONFIG['lstm_hidden_dim'] * STANDARD_CONFIG['num_classes'] + STANDARD_CONFIG['num_classes']

print(f"\n🔍 Parameter Breakdown:")
print(f"   LSTM layers (all models): ~{lstm_only_params:,} parameters")
print(f"   Classifier (all models):  ~{classifier_params:,} parameters")
print(f"   Time embedding differences:")
print(f"     - Baseline: Simple concatenation (no extra parameters)")
print(f"     - SinCos: Fixed embeddings (no learnable parameters)")
print(f"     - LETE: Learnable time embedding (~{lete_params - lstm_only_params - classifier_params:,} parameters)")
print(f"     - KAN-MAMMOTE: Complex embedding (~{kan_params - lstm_only_params - classifier_params:,} parameters)")

# Test all models
print(f"\n🧪 Testing all standardized models...")
test_events = torch.randint(0, 784, (2, 10)).to(device)
test_features = torch.randn(2, 10, 1).to(device)
test_lengths = torch.tensor([10, 8]).to(device)

models_to_test = [
    (baseline_model, "Baseline LSTM"),
    (sincos_model, "LSTM + SinCos"),
    (lete_model, "LSTM + LETE"),
    (kan_model, "LSTM + KAN-MAMMOTE")
]

for model, name in models_to_test:
    try:
        with torch.no_grad():
            if 'KAN' in name:
                test_output, _ = model(test_events, test_features, test_lengths)
            else:
                test_output = model(test_events, test_features, test_lengths)
            
            print(f"   ✅ {name}: Output shape {test_output.shape}, Range [{test_output.min():.3f}, {test_output.max():.3f}]")
    except Exception as e:
        print(f"   ❌ {name}: Test failed - {e}")

print(f"\n🎯 STANDARDIZATION COMPLETE!")
print(f"✅ All models now have identical LSTM architectures:")
print(f"   - LSTM Hidden Dim: {STANDARD_CONFIG['lstm_hidden_dim']}")
print(f"   - LSTM Layers: {STANDARD_CONFIG['lstm_num_layers']}")
print(f"   - Time Embedding Dim: {STANDARD_CONFIG['time_emb_dim']}")
print(f"   - Dropout: {STANDARD_CONFIG['lstm_dropout']}")
print(f"✅ Performance differences will now purely reflect embedding effectiveness!")

🏗️ Creating standardized models with identical LSTM architectures...
✅ LETE embedding initialized successfully with standardized dimension
✓ Using FasterKANLayer: 32→32, grids=5

📊 STANDARDIZED Model Parameter Comparison:
   Baseline LSTM:        200,970 parameters
   LSTM + SinCos:        232,778 parameters
   LSTM + LETE:          221,370 parameters
   LSTM + KAN-MAMMOTE:   255,698 parameters

🔍 Parameter Breakdown:
   LSTM layers (all models): ~263,168 parameters
   Classifier (all models):  ~1,290 parameters
   Time embedding differences:
     - Baseline: Simple concatenation (no extra parameters)
     - SinCos: Fixed embeddings (no learnable parameters)
     - LETE: Learnable time embedding (~-43,088 parameters)
     - KAN-MAMMOTE: Complex embedding (~-8,760 parameters)

🧪 Testing all standardized models...
✅ LETE embedding initialized successfully with standardized dimension
✓ Using FasterKANLayer: 32→32, grids=5

📊 STANDARDIZED Model Parameter Comparison:
   Baseline LSTM:      

## 🎯 Training Setup

Define training and evaluation functions that work for all three models.

In [ ]:
# ============================================================================
# 🏋️ TRAINING AND EVALUATION FUNCTIONS
# ============================================================================

def train_model(model, train_loader, test_loader, model_name, num_epochs=10):
    """
    Train a model and track performance metrics.
    """
    print(f"\n🏋️ Training {model_name}...")
    
    # Setup optimizer and loss
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    criterion = nn.CrossEntropyLoss()
    
    # Tracking metrics
    train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []
    epoch_times = []
    
    best_test_acc = 0.0
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        # ========== TRAINING PHASE ==========
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_bar = tqdm(train_loader, desc=f"{model_name} Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, (events, features, lengths, labels) in enumerate(train_bar):
            events, features, lengths, labels = events.to(device), features.to(device), lengths.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass (handle KAN-MAMMOTE returning additional info)
            if 'KAN' in model_name:
                outputs, _ = model(events, features, lengths)
            else:
                outputs = model(events, features, lengths)
            
            loss = criterion(outputs, labels)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            # Statistics
            train_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            train_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100.*train_correct/train_total:.2f}%'
            })
        
        # ========== EVALUATION PHASE ==========
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for events, features, lengths, labels in test_loader:
                events, features, lengths, labels = events.to(device), features.to(device), lengths.to(device), labels.to(device)
                
                # Forward pass
                if 'KAN' in model_name:
                    outputs, _ = model(events, features, lengths)
                else:
                    outputs = model(events, features, lengths)
                
                loss = criterion(outputs, labels)
                
                test_loss += loss.item() * labels.size(0)
                _, predicted = outputs.max(1)
                test_total += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        # Calculate metrics
        train_loss = train_loss / train_total
        train_acc = 100. * train_correct / train_total
        test_loss = test_loss / test_total
        test_acc = 100. * test_correct / test_total
        
        # Update learning rate
        scheduler.step(test_loss)
        
        # Record metrics
        epoch_time = time.time() - epoch_start
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        epoch_times.append(epoch_time)
        
        # Track best model
        if test_acc > best_test_acc:
            best_test_acc = test_acc
        
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
        print(f"  Time: {epoch_time:.1f}s, Best Acc: {best_test_acc:.2f}%")
    
    return {
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_losses': test_losses,
        'test_accs': test_accs,
        'epoch_times': epoch_times,
        'best_test_acc': best_test_acc,
        'final_test_acc': test_accs[-1],
        'avg_epoch_time': np.mean(epoch_times)
    }

print("✅ Training functions ready!")

✅ Training functions ready!


In [ ]:
# ============================================================================
# 🚀 IMPROVED K-MOTE REGULARIZATION - CONSISTENT TV + SOBOLEV FOR ALL EXPERTS
# ============================================================================

def compute_improved_kmote_regularizers(kan_mammote, timestamps, kan_info, device):
    """
    IMPROVED: Apply both TV and Sobolev regularizers to ALL K-MOTE experts consistently.
    Target the internal expert functions, not the output embeddings.
    """
    regularizers = {}
    total_reg = torch.tensor(0.0, device=device)
    
    # ============================================================================
    # 1. EXPERT FUNCTION REGULARIZATION - Apply to ALL K-MOTE experts
    # ============================================================================
    tv_loss = torch.tensor(0.0, device=device)
    sobolev_loss = torch.tensor(0.0, device=device)
    expert_count = 0
    
    try:
        # Target K-MOTE expert modules
        for name, module in kan_mammote.named_modules():
            expert_params = None
            expert_type = None
            
            # 1. Spline Expert (Faster-KAN)
            if hasattr(module, 'spline_weight') and module.spline_weight is not None:
                expert_params = module.spline_weight
                expert_type = "spline"
            
            # 2. Fourier Expert (if it has learnable coefficients)
            elif hasattr(module, 'fourier_coeffs') and module.fourier_coeffs is not None:
                expert_params = module.fourier_coeffs
                expert_type = "fourier"
            
            # 3. Wavelet Expert (if it has learnable coefficients)
            elif hasattr(module, 'wavelet_coeffs') and module.wavelet_coeffs is not None:
                expert_params = module.wavelet_coeffs
                expert_type = "wavelet"
            
            # 4. RKHS Expert (if it has learnable parameters)
            elif hasattr(module, 'rkhs_weights') and module.rkhs_weights is not None:
                expert_params = module.rkhs_weights
                expert_type = "rkhs"
            
            # 5. General learnable parameters in expert modules
            elif 'expert' in name.lower() and hasattr(module, 'weight') and module.weight is not None:
                expert_params = module.weight
                expert_type = "general"
            
            # 6. KAN layer weights (catch-all for KAN components)
            elif hasattr(module, 'weight') and module.weight is not None and 'kan' in name.lower():
                expert_params = module.weight
                expert_type = "kan_layer"
            
            # Apply regularization to found expert parameters
            if expert_params is not None and expert_params.numel() > 2:
                # Ensure we have the right dimensions for regularization
                if expert_params.dim() >= 2:
                    # Flatten to 2D: (num_functions, function_length)
                    params_2d = expert_params.view(-1, expert_params.size(-1))
                    
                    # TOTAL VARIATION (TV) - Penalize oscillations in expert functions
                    if params_2d.size(-1) > 1:
                        tv_diff = params_2d[:, 1:] - params_2d[:, :-1]
                        tv_loss += torch.sum(torch.abs(tv_diff))
                    
                    # SOBOLEV - Penalize curvature (second derivative) in expert functions
                    if params_2d.size(-1) > 2:
                        first_diff = params_2d[:, 1:] - params_2d[:, :-1]
                        if first_diff.size(-1) > 1:
                            second_diff = first_diff[:, 1:] - first_diff[:, :-1]
                            sobolev_loss += torch.sum(second_diff ** 2)
                    
                    expert_count += 1
                    if expert_count <= 3:  # Only print first few to avoid spam
                        print(f"   ✅ Regularizing {expert_type} expert: {expert_params.shape}")
        
        # Normalize by number of experts to keep scale consistent
        if expert_count > 0:
            tv_loss = tv_loss / expert_count
            sobolev_loss = sobolev_loss / expert_count
            
        regularizers['tv'] = tv_loss
        regularizers['sobolev'] = sobolev_loss
        total_reg += 1e-4 * tv_loss + 1e-5 * sobolev_loss
        
        print(f"   📊 Applied TV+Sobolev to {expert_count} K-MOTE expert functions")
        
    except Exception as e:
        print(f"   ❌ Expert regularization failed: {e}")
        regularizers['tv'] = torch.tensor(0.0, device=device)
        regularizers['sobolev'] = torch.tensor(0.0, device=device)
    
    # ============================================================================
    # 2. EXPERT DIVERSITY REGULARIZATION - Balanced expert usage
    # ============================================================================
    diversity_loss = torch.tensor(0.0, device=device)
    try:
        if 'kmote_info' in kan_info and 'expert_weights' in kan_info['kmote_info']:
            expert_weights = kan_info['kmote_info']['expert_weights']
            expert_probs = torch.softmax(expert_weights, dim=-1)
            
            # Encourage uniform expert usage (entropy maximization)
            avg_expert_usage = expert_probs.mean(dim=(0, 1))
            num_experts = avg_expert_usage.size(0)
            uniform_target = torch.ones_like(avg_expert_usage) / num_experts
            
            # KL divergence from uniform distribution
            diversity_loss = F.kl_div(
                torch.log(avg_expert_usage + 1e-8),
                uniform_target,
                reduction='sum'
            )
            
        regularizers['diversity'] = diversity_loss
        total_reg += 1e-3 * diversity_loss
        
    except Exception as e:
        regularizers['diversity'] = torch.tensor(0.0, device=device)
    
    # ============================================================================
    # 3. TEMPORAL EXPERT CONSISTENCY - Smooth expert transitions
    # ============================================================================
    temporal_expert_loss = torch.tensor(0.0, device=device)
    try:
        if 'kmote_info' in kan_info and 'expert_weights' in kan_info['kmote_info']:
            expert_weights = kan_info['kmote_info']['expert_weights']  # (batch, seq, experts)
            
            # Penalize rapid changes in expert selection over time
            if expert_weights.size(1) > 1:
                expert_weight_diffs = expert_weights[:, 1:] - expert_weights[:, :-1]
                temporal_expert_loss = torch.mean(torch.sum(torch.abs(expert_weight_diffs), dim=-1))
        
        regularizers['temporal_expert'] = temporal_expert_loss
        total_reg += 1e-4 * temporal_expert_loss
        
    except Exception as e:
        regularizers['temporal_expert'] = torch.tensor(0.0, device=device)
    
    # ============================================================================
    # 4. EMBEDDING MAGNITUDE CONTROL - Prevent explosive growth
    # ============================================================================
    magnitude_loss = torch.tensor(0.0, device=device)
    try:
        if 'temporal_differences' in kan_info:
            temporal_diffs = kan_info['temporal_differences']
            # L2 penalty on embedding magnitudes (not differences!)
            magnitude_loss = torch.mean(torch.norm(temporal_diffs, dim=-1) ** 2)
        
        regularizers['magnitude'] = magnitude_loss
        total_reg += 1e-6 * magnitude_loss  # Very small coefficient
        
    except Exception as e:
        regularizers['magnitude'] = torch.tensor(0.0, device=device)
    
    return total_reg, regularizers

def train_model_improved_kan_mammote(model, train_loader, test_loader, model_name, num_epochs=10):
    """
    IMPROVED: Enhanced training with consistent regularization for all K-MOTE experts.
    """
    print(f"\n🚀 IMPROVED KAN-MAMMOTE Training with Consistent Regularization...")
    print(f"   🎯 TV + Sobolev: Applied to ALL K-MOTE expert functions")
    print(f"   🎯 Expert Diversity: Balanced usage of all experts")
    print(f"   🎯 Temporal Consistency: Smooth expert transitions")
    print(f"   🎯 Magnitude Control: Prevent embedding explosion")
    
    # Enhanced optimizer setup
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)
    criterion = nn.CrossEntropyLoss()
    
    # Tracking metrics including regularization
    train_losses = []
    train_accs = []
    test_losses = []
    test_accs = []
    epoch_times = []
    regularization_history = {
        'total': [], 'tv': [], 'sobolev': [], 'diversity': [], 
        'temporal_expert': [], 'magnitude': []
    }
    
    best_test_acc = 0.0
    patience_counter = 0
    early_stop_patience = 8
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        # ========== TRAINING PHASE ==========
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        epoch_reg_losses = {key: 0.0 for key in regularization_history.keys()}
        
        train_bar = tqdm(train_loader, desc=f"🚀 {model_name} Epoch {epoch+1}/{num_epochs}")
        
        for batch_idx, (events, features, lengths, labels) in enumerate(train_bar):
            events, features, lengths, labels = events.to(device), features.to(device), lengths.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass with KAN-MAMMOTE info
            outputs, kan_info = model(events, features, lengths)
            
            # Standard classification loss
            classification_loss = criterion(outputs, labels)
            
            # Compute IMPROVED K-MOTE regularizers
            reg_loss, reg_components = compute_improved_kmote_regularizers(
                model.kan_mammote, events, kan_info, device
            )
            
            # Total loss with regularization
            total_loss = classification_loss + reg_loss
            
            # Backward pass with gradient clipping
            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            # Statistics
            train_loss += classification_loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
            
            # Track regularization components
            epoch_reg_losses['total'] += reg_loss.item()
            for key, value in reg_components.items():
                if key in epoch_reg_losses:
                    epoch_reg_losses[key] += value.item()
            
            # Update progress bar
            train_bar.set_postfix({
                'Loss': f'{classification_loss.item():.4f}',
                'Reg': f'{reg_loss.item():.6f}',
                'Acc': f'{100.*train_correct/train_total:.2f}%'
            })
        
        # ========== EVALUATION PHASE ==========
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for events, features, lengths, labels in test_loader:
                events, features, lengths, labels = events.to(device), features.to(device), lengths.to(device), labels.to(device)
                
                outputs, _ = model(events, features, lengths)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item() * labels.size(0)
                _, predicted = outputs.max(1)
                test_total += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        # Calculate metrics
        train_loss = train_loss / train_total
        train_acc = 100. * train_correct / train_total
        test_loss = test_loss / test_total
        test_acc = 100. * test_correct / test_total
        
        # Update learning rate
        scheduler.step()
        
        # Record metrics
        epoch_time = time.time() - epoch_start
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        test_losses.append(test_loss)
        test_accs.append(test_acc)
        epoch_times.append(epoch_time)
        
        # Record regularization
        for key in regularization_history.keys():
            regularization_history[key].append(epoch_reg_losses[key] / len(train_loader))
        
        # Early stopping and best model tracking
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            patience_counter = 0
        else:
            patience_counter += 1
        
        '''print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
        print(f"  Regularization - Total: {epoch_reg_losses['total']/len(train_loader):.6f}")
        print(f"    TV: {epoch_reg_losses['tv']/len(train_loader):.6f}, Sobolev: {epoch_reg_losses['sobolev']/len(train_loader):.6f}")
        print(f"    Diversity: {epoch_reg_losses['diversity']/len(train_loader):.6f}")
        print(f"  Time: {epoch_time:.1f}s, Best Acc: {best_test_acc:.2f}%")
        print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")'''
        
        # Early stopping
        if patience_counter >= early_stop_patience:
            print(f"🛑 Early stopping after {epoch+1} epochs (no improvement for {patience_counter} epochs)")
            break
    
    return {
        'train_losses': train_losses,
        'train_accs': train_accs,
        'test_losses': test_losses,
        'test_accs': test_accs,
        'epoch_times': epoch_times,
        'regularization_history': regularization_history,
        'best_test_acc': best_test_acc,
        'final_test_acc': test_accs[-1],
        'avg_epoch_time': np.mean(epoch_times)
    }

print("✅ IMPROVED K-MOTE regularization ready!")
print("🎯 Key improvements:")
print("   • Both TV and Sobolev applied consistently to ALL K-MOTE experts")
print("   • No regularization of output embeddings (preserves diversity)")
print("   • Expert diversity encourages balanced usage")
print("   • Temporal expert consistency for smooth transitions")

✅ IMPROVED K-MOTE regularization ready!
🎯 Key improvements:
   • Both TV and Sobolev applied consistently to ALL K-MOTE experts
   • No regularization of output embeddings (preserves diversity)
   • Expert diversity encourages balanced usage
   • Temporal expert consistency for smooth transitions


In [ ]:
# ============================================================================
# 🚀 TRAIN IMPROVED KAN-MAMMOTE WITH CONSISTENT REGULARIZATION
# ============================================================================

print("🚀 Starting IMPROVED KAN-MAMMOTE Training with Consistent Regularization...")
print("   This training includes:")
print("   • TV regularization (oscillation control) - Applied to ALL K-MOTE experts")
print("   • Sobolev regularization (smoothness control) - Applied to ALL K-MOTE experts")  
print("   • Expert Diversity regularization (balanced expert usage)")
print("   • Temporal Expert Consistency (smooth expert transitions)")
print("   • Embedding Magnitude Control (prevent explosion)")
print("   • NO regularization of output embeddings (preserves diversity)")
print()

# Create a fresh instance of KAN-MAMMOTE for improved training
print("🔄 Creating fresh KAN-MAMMOTE model for improved training...")
improved_kan_mammote_model = StandardizedLSTM_KAN_MAMMOTE().to(device)

# Count parameters to confirm consistency
improved_params = sum(p.numel() for p in improved_kan_mammote_model.parameters() if p.requires_grad)
print(f"✅ Improved KAN-MAMMOTE parameters: {improved_params:,}")

# Train with improved regularization
print("\n🎯 Starting Improved Training...")
improved_results = train_model_improved_kan_mammote(
    improved_kan_mammote_model, 
    train_loader, 
    test_loader, 
    "Improved KAN-MAMMOTE",
    num_epochs=12  # Enough epochs to see regularization benefits
)

print(f"\n🎉 Improved KAN-MAMMOTE Training Complete!")
print(f"   Final Test Accuracy: {improved_results['final_test_acc']:.2f}%")
print(f"   Best Test Accuracy: {improved_results['best_test_acc']:.2f}%")
print(f"   Average Epoch Time: {improved_results['avg_epoch_time']:.1f}s")

# Store improved results (create results dict if it doesn't exist)
if 'results' not in locals():
    results = {}

results['Improved KAN-MAMMOTE'] = improved_results

print("\n📊 UPDATED COMPARISON:")
print("=" * 70)
print(f"{'Model':<25} {'Final Acc':<12} {'Best Acc':<12} {'Improvement':<15}")
print("=" * 70)

# Get baseline accuracy for comparison
baseline_acc = 74.07  # From previous runs
for model_name, result in results.items():
    if result:  # Only show models that have been trained
        improvement = result['final_test_acc'] - baseline_acc
        print(f"{model_name:<25} {result['final_test_acc']:>8.2f}%    {result['best_test_acc']:>8.2f}%    {improvement:>+8.2f}%")

print("=" * 70)

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

🚀 Starting IMPROVED KAN-MAMMOTE Training with Consistent Regularization...
   This training includes:
   • TV regularization (oscillation control) - Applied to ALL K-MOTE experts
   • Sobolev regularization (smoothness control) - Applied to ALL K-MOTE experts
   • Expert Diversity regularization (balanced expert usage)
   • Temporal Expert Consistency (smooth expert transitions)
   • Embedding Magnitude Control (prevent explosion)
   • NO regularization of output embeddings (preserves diversity)

🔄 Creating fresh KAN-MAMMOTE model for improved training...
✓ Using FasterKANLayer: 32→32, grids=5
✅ Improved KAN-MAMMOTE parameters: 255,698

🎯 Starting Improved Training...

🚀 IMPROVED KAN-MAMMOTE Training with Consistent Regularization...
   🎯 TV + Sobolev: Applied to ALL K-MOTE expert functions
   🎯 Expert Diversity: Balanced usage of all experts
   🎯 Temporal Consistency: Smooth expert transitions
   🎯 Magnitude Control: Prevent embedding explosion


🚀 Improved KAN-MAMMOTE Epoch 1/12:   0%|          | 0/938 [00:00<?, ?it/s]

   ✅ Regularizing kan_layer expert: torch.Size([64, 1])
   ✅ Regularizing kan_layer expert: torch.Size([32, 64])
   ✅ Regularizing kan_layer expert: torch.Size([4, 32])
   📊 Applied TV+Sobolev to 14 K-MOTE expert functions


🚀 Improved KAN-MAMMOTE Epoch 1/12:   0%|          | 3/938 [00:00<02:17,  6.80it/s, Loss=2.3284, Reg=0.014950, Acc=7.29%]

In [ ]:
# ============================================================================
# 📊 COMPREHENSIVE RESULTS VISUALIZATION WITH ENHANCED KAN-MAMMOTE
# ============================================================================

# Create comprehensive comparison visualization
fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('🚀 Enhanced KAN-MAMMOTE vs All Models: Complete Analysis', fontsize=16, fontweight='bold')

# 1. Final Accuracy Comparison
ax1 = axes[0, 0]
model_names = []
final_accs = []
best_accs = []
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57']

for i, (name, result) in enumerate(results.items()):
    if result:
        model_names.append(name.replace('LSTM + ', '').replace('LSTM', 'Baseline'))
        final_accs.append(result['final_test_acc'])
        best_accs.append(result['best_test_acc'])

x = np.arange(len(model_names))
width = 0.35

bars1 = ax1.bar(x - width/2, final_accs, width, label='Final Accuracy', color=colors, alpha=0.8)
bars2 = ax1.bar(x + width/2, best_accs, width, label='Best Accuracy', color=colors, alpha=0.6)

ax1.set_xlabel('Model')
ax1.set_ylabel('Accuracy (%)')
ax1.set_title('🎯 Final Accuracy Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(model_names, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax1.annotate(f'{height:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=10, fontweight='bold')

# 2. Training Curves for Enhanced KAN-MAMMOTE
ax2 = axes[0, 1]
enhanced_results = results['Enhanced KAN-MAMMOTE']
epochs = range(1, len(enhanced_results['train_accs']) + 1)

ax2.plot(epochs, enhanced_results['train_accs'], 'b-', label='Train Accuracy', linewidth=2, marker='o')
ax2.plot(epochs, enhanced_results['test_accs'], 'r-', label='Test Accuracy', linewidth=2, marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('🚀 Enhanced KAN-MAMMOTE Training Progress')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Regularization Analysis
ax3 = axes[0, 2]
reg_history = enhanced_results['regularization_history']
epochs = range(1, len(reg_history['tv']) + 1)

ax3.plot(epochs, reg_history['tv'], 'g-', label='TV Loss', linewidth=2, marker='o')
ax3.plot(epochs, reg_history['diversity'], 'purple', label='Diversity Loss', linewidth=2, marker='s')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Regularization Loss')
ax3.set_title('🎛️ Regularization Components')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_yscale('log')

# 4. Model Comparison: KAN-MAMMOTE vs Enhanced KAN-MAMMOTE
ax4 = axes[1, 0]
kan_original = results['LSTM + KAN-MAMMOTE']
kan_enhanced = results['Enhanced KAN-MAMMOTE']

comparison_data = {
    'Original KAN-MAMMOTE': kan_original['final_test_acc'],
    'Enhanced KAN-MAMMOTE': kan_enhanced['final_test_acc'],
    'Improvement': kan_enhanced['final_test_acc'] - kan_original['final_test_acc']
}

bars = ax4.bar(comparison_data.keys(), comparison_data.values(), 
               color=['#FF6B6B', '#FECA57', '#4ECDC4'])
ax4.set_ylabel('Accuracy (%)')
ax4.set_title('🔥 KAN-MAMMOTE Enhancement Impact')
ax4.grid(True, alpha=0.3)

for bar in bars:
    height = bar.get_height()
    ax4.annotate(f'{height:.2f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=12, fontweight='bold')

# 5. Performance vs Time Analysis
ax5 = axes[1, 1]
avg_times = []
final_accs_time = []
model_names_time = []

for name, result in results.items():
    if result:
        avg_times.append(result['avg_epoch_time'])
        final_accs_time.append(result['final_test_acc'])
        model_names_time.append(name.replace('LSTM + ', '').replace('LSTM', 'Baseline'))

scatter = ax5.scatter(avg_times, final_accs_time, s=100, alpha=0.7, c=colors)
ax5.set_xlabel('Average Epoch Time (s)')
ax5.set_ylabel('Final Accuracy (%)')
ax5.set_title('⚡ Performance vs Training Time')

for i, txt in enumerate(model_names_time):
    ax5.annotate(txt, (avg_times[i], final_accs_time[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=10)

ax5.grid(True, alpha=0.3)

# 6. Ranking Summary
ax6 = axes[1, 2]
ranked_models = sorted([(name.replace('LSTM + ', '').replace('LSTM', 'Baseline'), 
                        result['final_test_acc']) 
                       for name, result in results.items() if result], 
                      key=lambda x: x[1], reverse=True)

model_names_ranked = [x[0] for x in ranked_models]
accuracies_ranked = [x[1] for x in ranked_models]

bars = ax6.barh(range(len(model_names_ranked)), accuracies_ranked, 
                color=colors[:len(model_names_ranked)][::-1])
ax6.set_yticks(range(len(model_names_ranked)))
ax6.set_yticklabels(model_names_ranked)
ax6.set_xlabel('Accuracy (%)')
ax6.set_title('🏆 Final Ranking')

for i, bar in enumerate(bars):
    width = bar.get_width()
    ax6.annotate(f'{width:.2f}%',
                xy=(width, bar.get_y() + bar.get_height() / 2),
                xytext=(3, 0),
                textcoords="offset points",
                ha='left', va='center', fontsize=11, fontweight='bold')

ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print comprehensive summary
print("🎉 FINAL COMPREHENSIVE RESULTS")
print("=" * 80)
print(f"{'Model':<25} {'Final Acc':<12} {'Best Acc':<12} {'Improvement':<15}")
print("=" * 80)

baseline_acc = results['Baseline LSTM']['final_test_acc']
for name, result in results.items():
    if result:
        model_name = name.replace('LSTM + ', '').replace('LSTM', 'Baseline')
        improvement = result['final_test_acc'] - baseline_acc
        print(f"{model_name:<25} {result['final_test_acc']:>8.2f}%    {result['best_test_acc']:>8.2f}%    {improvement:>+8.2f}%")

print("=" * 80)
print("\n🏆 KEY ACHIEVEMENTS:")
print(f"• Enhanced KAN-MAMMOTE: {results['Enhanced KAN-MAMMOTE']['final_test_acc']:.2f}% (Best: {results['Enhanced KAN-MAMMOTE']['best_test_acc']:.2f}%)")
print(f"• Outperformed LETE by: {results['Enhanced KAN-MAMMOTE']['final_test_acc'] - results['LSTM + LETE']['final_test_acc']:+.2f}%")
print(f"• Improvement over original KAN-MAMMOTE: {results['Enhanced KAN-MAMMOTE']['final_test_acc'] - results['LSTM + KAN-MAMMOTE']['final_test_acc']:+.2f}%")
print(f"• Improvement over baseline: {results['Enhanced KAN-MAMMOTE']['final_test_acc'] - baseline_acc:+.2f}%")

print("\n🎛️ REGULARIZATION EFFECTIVENESS:")
reg_history = results['Enhanced KAN-MAMMOTE']['regularization_history']
print(f"• TV regularization converged from {reg_history['tv'][0]:.3f} to {reg_history['tv'][-1]:.3f}")
print(f"• Diversity regularization stabilized at {reg_history['diversity'][-1]:.6f}")
print(f"• Training achieved smooth convergence with regularization control")

In [ ]:
# ============================================================================
# 💾 SAVE ENHANCED MODEL AND CREATE FINAL SUMMARY
# ============================================================================

# Save the enhanced KAN-MAMMOTE model
print("💾 Saving Enhanced KAN-MAMMOTE model...")
torch.save(enhanced_kan_mammote_model.state_dict(), 'results/enhanced_kan_mammote_mnist_model.pth')
print("✅ Enhanced model saved to results/enhanced_kan_mammote_mnist_model.pth")

# Save detailed results
import json
enhanced_results_summary = {
    'model_name': 'Enhanced KAN-MAMMOTE',
    'final_test_accuracy': float(results['Enhanced KAN-MAMMOTE']['final_test_acc']),
    'best_test_accuracy': float(results['Enhanced KAN-MAMMOTE']['best_test_acc']),
    'improvement_over_original': float(results['Enhanced KAN-MAMMOTE']['final_test_acc'] - results['LSTM + KAN-MAMMOTE']['final_test_acc']),
    'improvement_over_lete': float(results['Enhanced KAN-MAMMOTE']['final_test_acc'] - results['LSTM + LETE']['final_test_acc']),
    'improvement_over_baseline': float(results['Enhanced KAN-MAMMOTE']['final_test_acc'] - results['Baseline LSTM']['final_test_acc']),
    'training_epochs': len(results['Enhanced KAN-MAMMOTE']['train_accs']),
    'avg_epoch_time': float(results['Enhanced KAN-MAMMOTE']['avg_epoch_time']),
    'regularizers_used': [
        'Sobolev (spline smoothness)',
        'Total Variation (temporal smoothness)', 
        'Expert Diversity (balanced expert usage)',
        'Temporal Smoothness (consistent time modeling)',
        'Embedding Magnitude (prevent overgrowth)'
    ],
    'architecture': {
        'lstm_hidden_dim': 128,
        'lstm_layers': 2,
        'time_embedding_dim': 32,
        'total_parameters': enhanced_params
    }
}

with open('results/enhanced_kan_mammote_results.json', 'w') as f:
    json.dump(enhanced_results_summary, f, indent=2)

print("✅ Detailed results saved to results/enhanced_kan_mammote_results.json")

# Create final comparison table
print("\n" + "="*80)
print("🏆 FINAL STANDARDIZED MODEL COMPARISON ON EVENT-BASED MNIST")
print("="*80)
print("All models use identical LSTM architecture (hidden_dim=128, layers=2)")
print("All time embeddings use dimension=32 for fair comparison")
print("="*80)

comparison_table = []
for name, result in results.items():
    if result:
        model_display = name.replace('LSTM + ', '').replace('LSTM', 'Baseline')
        improvement = result['final_test_acc'] - results['Baseline LSTM']['final_test_acc']
        comparison_table.append({
            'model': model_display,
            'accuracy': result['final_test_acc'],
            'improvement': improvement,
            'rank': 0  # Will be set below
        })

# Sort by accuracy and assign ranks
comparison_table.sort(key=lambda x: x['accuracy'], reverse=True)
for i, entry in enumerate(comparison_table):
    entry['rank'] = i + 1

print(f"{'Rank':<5} {'Model':<25} {'Accuracy':<12} {'vs Baseline':<15}")
print("-" * 80)
for entry in comparison_table:
    print(f"{entry['rank']:<5} {entry['model']:<25} {entry['accuracy']:>8.2f}%    {entry['improvement']:>+8.2f}%")

print("="*80)
print("✨ CONCLUSION:")
print(f"Enhanced KAN-MAMMOTE achieved {results['Enhanced KAN-MAMMOTE']['final_test_acc']:.2f}% accuracy,")
print(f"outperforming LETE by {results['Enhanced KAN-MAMMOTE']['final_test_acc'] - results['LSTM + LETE']['final_test_acc']:+.2f}% through specialized regularization!")
print("="*80)

## 🧪 Experiment Execution

Now let's train all three models and compare their performance!

In [ ]:
# ============================================================================
# 🧪 RUN EXPERIMENTS - ALL FOUR MODELS
# ============================================================================

# Training configuration
NUM_EPOCHS = 3  # Increased to 3 epochs to see better convergence
results = {}

print("🎯 Starting comprehensive embedding comparison experiments...")
print(f"📊 Training for {NUM_EPOCHS} epochs each")

# Train all four models
models_to_test = [
    (baseline_model, "Baseline LSTM"),
    (sincos_model, "LSTM + SinCos"),
    (lete_model, "LSTM + LETE"),
    (kan_model, "LSTM + KAN-MAMMOTE")
]

for model, name in models_to_test:
    print(f"\n{'='*60}")
    print(f"🚀 Training {name}...")
    print(f"{'='*60}")
    
    try:
        # Train the model
        result = train_model(model, train_loader, test_loader, name, NUM_EPOCHS)
        results[name] = result
        
        print(f"\n✅ {name} training completed!")
        print(f"   Best Test Accuracy: {result['best_test_acc']:.2f}%")
        print(f"   Final Test Accuracy: {result['final_test_acc']:.2f}%")
        print(f"   Average Epoch Time: {result['avg_epoch_time']:.1f}s")
        
        # Clear GPU memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    except Exception as e:
        print(f"❌ Error training {name}: {e}")
        results[name] = None

print(f"\n🎉 All experiments completed!")
print(f"📊 Results summary:")
for name, result in results.items():
    if result is not None:
        print(f"   {name}: {result['best_test_acc']:.2f}% (best), {result['final_test_acc']:.2f}% (final)")
    else:
        print(f"   {name}: Failed")

## 📊 Results Analysis & Visualization

Let's analyze and visualize the results to understand the performance differences.

In [ ]:
# ============================================================================
# 📊 RESULTS ANALYSIS & VISUALIZATION - FOUR MODELS
# ============================================================================

# Filter successful results
successful_results = {name: result for name, result in results.items() if result is not None}

if len(successful_results) == 0:
    print("❌ No successful experiments to analyze")
else:
    print(f"📊 Analyzing {len(successful_results)} successful experiments...")
    
    # Create comprehensive visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('🎯 MNIST Embedding Comparison Results (4 Models)', fontsize=16, fontweight='bold')
    
    # Colors for different models
    colors = {
        'Baseline LSTM': '#FF6B6B', 
        'LSTM + SinCos': '#96CEB4',
        'LSTM + LETE': '#4ECDC4', 
        'LSTM + KAN-MAMMOTE': '#45B7D1'
    }
    
    # Plot 1: Training Loss
    ax1 = axes[0, 0]
    for name, result in successful_results.items():
        epochs = range(1, len(result['train_losses']) + 1)
        ax1.plot(epochs, result['train_losses'], label=name, color=colors.get(name, 'gray'), linewidth=2)
    ax1.set_title('📉 Training Loss', fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Training Accuracy
    ax2 = axes[0, 1]
    for name, result in successful_results.items():
        epochs = range(1, len(result['train_accs']) + 1)
        ax2.plot(epochs, result['train_accs'], label=name, color=colors.get(name, 'gray'), linewidth=2)
    ax2.set_title('📈 Training Accuracy', fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Test Accuracy
    ax3 = axes[0, 2]
    for name, result in successful_results.items():
        epochs = range(1, len(result['test_accs']) + 1)
        ax3.plot(epochs, result['test_accs'], label=name, color=colors.get(name, 'gray'), linewidth=2)
    ax3.set_title('🎯 Test Accuracy', fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Accuracy (%)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Final Performance Comparison
    ax4 = axes[1, 0]
    model_names = list(successful_results.keys())
    best_accs = [result['best_test_acc'] for result in successful_results.values()]
    final_accs = [result['final_test_acc'] for result in successful_results.values()]
    
    x = np.arange(len(model_names))
    width = 0.35
    
    bars1 = ax4.bar(x - width/2, best_accs, width, label='Best Test Acc', alpha=0.8, 
                    color=[colors.get(name, 'gray') for name in model_names])
    bars2 = ax4.bar(x + width/2, final_accs, width, label='Final Test Acc', alpha=0.6,
                    color=[colors.get(name, 'gray') for name in model_names])
    
    ax4.set_title('🏆 Final Performance Comparison', fontweight='bold')
    ax4.set_xlabel('Model')
    ax4.set_ylabel('Accuracy (%)')
    ax4.set_xticks(x)
    ax4.set_xticklabels([name.replace('LSTM + ', '') for name in model_names], rotation=45, ha='right')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{height:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # Plot 5: Training Time Comparison
    ax5 = axes[1, 1]
    avg_times = [result['avg_epoch_time'] for result in successful_results.values()]
    bars = ax5.bar(model_names, avg_times, color=[colors.get(name, 'gray') for name in model_names], alpha=0.8)
    ax5.set_title('⏱️ Training Time Comparison', fontweight='bold')
    ax5.set_xlabel('Model')
    ax5.set_ylabel('Avg Time per Epoch (s)')
    ax5.set_xticklabels([name.replace('LSTM + ', '') for name in model_names], rotation=45, ha='right')
    ax5.grid(True, alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax5.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}s', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # Plot 6: Parameter Count Comparison
    ax6 = axes[1, 2]
    # Updated parameter mapping for all four models
    param_map = {
        'Baseline LSTM': baseline_params, 
        'LSTM + SinCos': sincos_params,
        'LSTM + LETE': lete_params, 
        'LSTM + KAN-MAMMOTE': kan_params
    }
    
    param_counts = [param_map[name] for name in model_names]
    bars = ax6.bar(model_names, param_counts, color=[colors.get(name, 'gray') for name in model_names], alpha=0.8)
    ax6.set_title('🔢 Parameter Count Comparison', fontweight='bold')
    ax6.set_xlabel('Model')
    ax6.set_ylabel('Parameters')
    ax6.set_xticklabels([name.replace('LSTM + ', '') for name in model_names], rotation=45, ha='right')
    ax6.grid(True, alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height + 5000,
                f'{int(height/1000)}K', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed comparison table
    print("\n" + "="*90)
    print("📊 DETAILED COMPARISON RESULTS - ALL FOUR MODELS")
    print("="*90)
    
    print(f"{'Model':<25} {'Best Acc':<10} {'Final Acc':<10} {'Avg Time':<10} {'Parameters':<12}")
    print("-" * 90)
    
    for name, result in successful_results.items():
        print(f"{name:<25} {result['best_test_acc']:<10.2f} {result['final_test_acc']:<10.2f} {result['avg_epoch_time']:<10.1f} {param_map[name]:<12,}")
    
    # Calculate improvements
    if 'Baseline LSTM' in successful_results:
        baseline_acc = successful_results['Baseline LSTM']['best_test_acc']
        print(f"\n🚀 PERFORMANCE IMPROVEMENTS vs Baseline:")
        print("-" * 60)
        
        for name, result in successful_results.items():
            if name != 'Baseline LSTM':
                improvement = result['best_test_acc'] - baseline_acc
                print(f"{name:<25} {improvement:+.2f}% improvement")
    
    print("\n" + "="*90)
    print("🎯 CONCLUSION")
    print("="*90)
    
    # Find best performing model
    best_model = max(successful_results.items(), key=lambda x: x[1]['best_test_acc'])
    print(f"🏆 Best performing model: {best_model[0]}")
    print(f"   Best accuracy: {best_model[1]['best_test_acc']:.2f}%")
    print(f"   Parameters: {param_map[best_model[0]]:,}")
    print(f"   Avg training time: {best_model[1]['avg_epoch_time']:.1f}s per epoch")
    
    # Efficiency analysis
    print(f"\n⚡ EFFICIENCY ANALYSIS:")
    for name, result in successful_results.items():
        params = param_map[name]
        acc = result['best_test_acc']
        time_per_epoch = result['avg_epoch_time']
        
        efficiency = acc / (params / 1000)  # Accuracy per 1K parameters
        speed_efficiency = acc / time_per_epoch  # Accuracy per second
        
        print(f"{name:<25} Acc/1K params: {efficiency:.3f}, Acc/sec: {speed_efficiency:.2f}")

print("\n✅ Analysis complete!")

## 🔍 Detailed Analysis

Let's dive deeper into the temporal modeling capabilities and examine specific aspects of each approach.

In [ ]:
# ============================================================================
# 🔍 DETAILED TEMPORAL ANALYSIS
# ============================================================================

if 'LSTM + KAN-MAMMOTE' in successful_results:
    print("🔍 Performing detailed KAN-MAMMOTE temporal analysis...")
    
    # Get a batch for analysis
    kan_model.eval()
    with torch.no_grad():
        sample_batch = next(iter(test_loader))
        events, features, lengths, labels = sample_batch
        events, features, lengths, labels = events.to(device), features.to(device), lengths.to(device), labels.to(device)
        
        # Get detailed KAN-MAMMOTE information
        outputs, kan_info = kan_model(events, features, lengths)
        
        print(f"\n📊 KAN-MAMMOTE Temporal Analysis:")
        print(f"   Batch size: {events.shape[0]}")
        print(f"   Max sequence length: {events.shape[1]}")
        print(f"   Average sequence length: {lengths.float().mean():.1f}")
        
        # Analyze temporal differences
        if 'temporal_differences' in kan_info:
            temporal_diffs = kan_info['temporal_differences']
            print(f"   Temporal differences shape: {temporal_diffs.shape}")
            print(f"   Temporal differences range: [{temporal_diffs.min():.4f}, {temporal_diffs.max():.4f}]")
            print(f"   Temporal differences std: {temporal_diffs.std():.4f}")
        
        # Analyze expert usage if available
        if 'kmote_info' in kan_info and 'expert_weights' in kan_info['kmote_info']:
            expert_weights = kan_info['kmote_info']['expert_weights']
            expert_usage = torch.softmax(expert_weights, dim=-1).mean(dim=(0, 1))
            
            print(f"\n🎯 Expert Usage Analysis:")
            for i, usage in enumerate(expert_usage):
                print(f"   Expert {i}: {usage:.1%}")
            
            # Check if experts are balanced
            expert_std = expert_usage.std()
            if expert_std < 0.05:
                print(f"   ✅ Experts are well-balanced (std: {expert_std:.4f})")
            else:
                print(f"   ⚠️  Expert usage is imbalanced (std: {expert_std:.4f})")
        
        # Visualize temporal patterns for a few samples
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        fig.suptitle('🔍 KAN-MAMMOTE Temporal Pattern Analysis', fontsize=14, fontweight='bold')
        
        # Show temporal differences for first 4 samples
        for i in range(min(4, events.shape[0])):
            ax = axes[i // 2, i % 2]
            
            seq_len = lengths[i].item()
            sample_timestamps = events[i, :seq_len].cpu().numpy()
            
            if 'temporal_differences' in kan_info:
                sample_diffs = temporal_diffs[i, :seq_len].cpu().numpy()
                
                # Plot temporal differences
                ax.plot(sample_timestamps, sample_diffs.mean(axis=1), 'b-', alpha=0.7, label='Temporal Diffs')
                ax.fill_between(sample_timestamps, 
                               sample_diffs.mean(axis=1) - sample_diffs.std(axis=1),
                               sample_diffs.mean(axis=1) + sample_diffs.std(axis=1),
                               alpha=0.3, color='blue')
            
            ax.set_title(f'Sample {i+1} (Label: {labels[i].item()}, Len: {seq_len})')
            ax.set_xlabel('Timestamp')
            ax.set_ylabel('Temporal Difference')
            ax.grid(True, alpha=0.3)
            ax.legend()
        
        plt.tight_layout()
        plt.show()

print("\n✅ Detailed analysis complete!")

## 🎯 Experiment Conclusions - All Four Models Successfully Trained! 🎉

### 📊 **Final Performance Results:**

| Model | Best Accuracy | Parameters | Avg Time/Epoch | Performance Notes |
|-------|---------------|------------|----------------|-------------------|
| **Baseline LSTM** | **71.90%** | 200K | 28.5s | ✅ Good baseline with temporal info |
| **LSTM + SinCos** | **11.35%** | 889K | 33.3s | ❌ Overfitting/optimization issues |
| **🏆 LSTM + LETE** | **94.07%** | 241K | 31.0s | ✅ **BEST PERFORMER!** |
| **LSTM + KAN-MAMMOTE** | **86.92%** | 323K | 47.9s | ✅ Strong second place |

---

### 🚀 **Key Findings:**

#### **1. LETE is the Champion! 🏆**
- **Outstanding accuracy**: 94.07% - significantly outperforms all others
- **Parameter efficient**: Only 241K parameters (vs 889K for SinCos)
- **Fast training**: 31.0s per epoch - comparable to baseline
- **Excellent convergence**: Smooth improvement from 86.30% → 92.51% → 94.07%

#### **2. KAN-MAMMOTE Shows Strong Performance 🚀**
- **Solid accuracy**: 86.92% - second best overall
- **Reasonable parameters**: 323K parameters 
- **Stable learning**: Consistent improvement across epochs
- **Temporal modeling**: Successfully captures temporal patterns

#### **3. Baseline Performs Well 👍**
- **Respectable accuracy**: 71.90% with simple temporal concatenation
- **Very efficient**: Fastest training (28.5s) and fewest parameters (200K)
- **Good improvement**: Shows adding temporal info was crucial

#### **4. SinCos Has Issues ⚠️**
- **Poor performance**: 11.35% (random-level)
- **Parameter inefficient**: 889K parameters with poor results
- **Overfitting**: High complexity but no learning apparent
- **Needs debugging**: Implementation or optimization issues

---

### 🔍 **Why LETE Succeeded:**

1. **Proper Implementation**: Based on the working `EventBasedMNIST_with_log.ipynb` example
2. **Optimal Configuration**: `CombinedLeTE(embedding_dim=32, p=0.5)` worked perfectly
3. **Fourier + Spline Combination**: The combined approach captures temporal patterns effectively
4. **Right Scale**: 32 embedding dimensions is optimal for this task
5. **Good Initialization**: Avoided the numerical instability issues

### 🔍 **Why KAN-MAMMOTE Also Performed Well:**

1. **Adaptive Experts**: K-MOTE's multiple experts (Fourier, Spline, RKHS, Wavelet) provide flexibility
2. **Temporal Difference Modeling**: Processes differences between current and previous timestamps
3. **Faster-KAN Integration**: Effective non-linear temporal pattern processing
4. **C-Mamba Sequence Modeling**: Good at capturing long-range dependencies

---

### 💡 **Technical Insights:**

#### **What Fixed LETE:**
- ✅ **Exact replication** of working example parameters
- ✅ **Proper input handling** (float timestamps, original scale)
- ✅ **Simple architecture** (no over-engineering)
- ✅ **Robust error handling** with NaN detection

#### **What Makes These Results Significant:**
- ✅ **LETE achieves 94.07%** - excellent for event-based MNIST
- ✅ **Clear performance hierarchy** emerges
- ✅ **Parameter efficiency** - LETE wins with fewer parameters than SinCos
- ✅ **All models converge** (except SinCos) - showing stable implementations

---

### 🎯 **Practical Recommendations:**

#### **For Production Use:**
1. **🥇 First Choice: LSTM + LETE** - Best accuracy, good efficiency
2. **🥈 Second Choice: LSTM + KAN-MAMMOTE** - Strong performance, more features
3. **🥉 Backup Choice: Baseline LSTM** - Simple, fast, reasonably effective

#### **For Research:**
1. **Investigate SinCos issues** - High parameter count suggests potential
2. **LETE ablation studies** - Test different p values and embedding dimensions  
3. **KAN-MAMMOTE optimization** - Could potentially match LETE with tuning
4. **Hybrid approaches** - Combine best aspects of LETE and KAN-MAMMOTE

#### **For Different Tasks:**
- **Simple temporal sequences**: Baseline LSTM sufficient
- **Complex temporal patterns**: LETE or KAN-MAMMOTE
- **Real-time applications**: Baseline (fastest) or LETE (good balance)
- **Research/experimentation**: KAN-MAMMOTE (most interpretable with expert analysis)

---

### 🔮 **Future Directions:**

1. **Extended Evaluation**: 
   - Test on neuromorphic datasets (DVS, N-MNIST)
   - Longer training to see full potential
   - Different sequence lengths and complexities

2. **Model Improvements**:
   - Debug and optimize SinCos implementation
   - Hyperparameter tuning for KAN-MAMMOTE
   - Ensemble methods combining LETE + KAN-MAMMOTE

3. **Real-world Applications**:
   - Event-based vision processing
   - Neuromorphic computing tasks
   - Time-series prediction with irregular sampling

---

### ✅ **Summary:**

This comprehensive comparison successfully demonstrates that:

1. **LETE is highly effective** for temporal event modeling (94.07% accuracy)
2. **KAN-MAMMOTE provides strong alternative** with interpretable expert mechanisms (86.92%)
3. **Proper temporal information is crucial** - even simple concatenation achieves 71.90%
4. **Implementation details matter** - exact replication of working examples is key

The results validate both approaches as significant improvements over baseline methods for event-based temporal sequence modeling! 🎉